In [1]:
from transformers import Qwen2ForCausalLM, AutoTokenizer
import torch
from copy import deepcopy
from tqdm import tqdm

/root/miniconda3/envs/pissa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Qwen2ForCausalLM.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-3B", device_map='cuda:2', attn_implementation="sdpa")
tokenizer = AutoTokenizer.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-3B")
model

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.36s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm):

In [3]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:2"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。每天，它都在森林中飞来飞去，寻找可以交朋友的伙伴。

一天，蓝羽遇到了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽立刻停下来，用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个好伙伴。”

从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。蓝羽教会了米洛如何飞翔，而米洛则教给了蓝羽如何照顾受伤的动物。

随着时间的推移，森林里的其他动物也开始注意到蓝羽和米洛之间的特殊关系。它们邀请蓝羽和米洛参加各种活动，包括音乐会、舞蹈比赛和野餐。蓝羽和米洛都非常开心，因为他们知道，有了朋友，生活变得更加美好。

然而，好景不长，森林里来了一只凶猛的狼，它威胁着所有动物的安全。蓝羽和米洛决定联手对抗这只狼。他们利用自己的智慧和勇气，成功地将狼赶走了。

从那以后，蓝羽和米洛成为了森林中的英雄。他们的友谊证明了，即使是最小的生物也能拥有巨大的力量和勇气。而蓝羽也明白了，真正的友谊是无价的，它能够带来快乐、支持和力量。

这个故事告诉我们，无论我们多么渺小，只要我们有勇气去帮助别人，就有机会找到真正的友谊和幸福。<|endoftext|>


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"

In [5]:
from vllm import LLM, SamplingParams
llm = LLM("/root/mfx/huggingface/Qwen/Qwen2.5-3B", tensor_parallel_size=1, trust_remote_code=True)

2025-02-19 00:43:25,870	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 02-19 00:43:26 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='/root/mfx/huggingface/Qwen/Qwen2.5-3B', speculative_config=None, tokenizer='/root/mfx/huggingface/Qwen/Qwen2.5-3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/root/mfx/huggingface/Qwen/Qwen2.5-3B, use_v2_block_manager=False, num_scheduler

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.03s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.23it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.18it/s]


INFO 02-19 00:43:29 model_runner.py:1025] Loading model weights took 5.7915 GB


INFO 02-19 00:43:30 gpu_executor.py:122] # GPU blocks: 48781, # CPU blocks: 7281
INFO 02-19 00:43:32 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-19 00:43:32 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-19 00:43:43 model_runner.py:1456] Graph capturing finished in 11 secs.


In [8]:
prompts = [
    "给我讲一个故事吧",
]
sampling_params = SamplingParams(temperature=0, max_tokens=512)
outputs = llm.generate(prompts, sampling_params)
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.70s/it, est. speed input: 1.85 toks/s, output: 120.52 toks/s]

Prompt: '给我讲一个故事吧', Generated text: '。\n当然可以，这是一个关于勇气和友谊的故事。\n\n从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。每天，它都在森林中飞来飞去，寻找可以交朋友的伙伴。\n\n一天，蓝羽遇到了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽立刻停下来，用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个好伙伴。”\n\n从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。蓝羽教会了米洛如何飞翔，而米洛则教给了蓝羽如何照顾受伤的动物。\n\n然而，好景不长，森林里来了一只凶猛的狼，它威胁着所有动物的安全。蓝羽和米洛决定联合起来，保护他们的家园。他们一起制定了一个计划，用智慧和勇气对抗狼群。\n\n在一次激烈的战斗中，蓝羽和米洛展现出了惊人的勇气。它们用它们的翅膀和尖锐的喙，成功地击退了狼群。森林里的动物们欢呼雀跃，庆祝他们的胜利。\n\n从那以后，蓝羽和米洛成为了森林中的英雄。他们的友谊证明了，即使是最小的生物也能拥有巨大的力量和勇气。而蓝羽也学会了，真正的友谊是无价的，它能够超越一切困难和挑战。'
